In [2]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import platform
plt.style.use('seaborn')
from datetime import datetime
import json

from metric import E1_loss, E2_loss, total_loss
from models import classifier, cnn_model, conv_block, cnn_parallel
from utils import train_model, eval_model, dfDataset, weights_init

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from sklearn.model_selection import KFold, train_test_split
from torchsummary import summary

- parallel cnn 구현
- lstm

### Configuration

In [3]:
EPOCH = 100
base_lr = 0.001
now = datetime.strftime(datetime.now(), '%Y%m%d-%H%M%S')
save_path = './model/{}'.format(now)
initialize = True
print_summary = True
batch_size = 256
nfold = 10

### load dataset

In [4]:
if platform.system() == 'Windows':
    root_dir = 'D:/datasets/KAERI_dataset/'
else:
    root_dir = '/home/bskim/project/kaeri/KAERI_dataset/'

train_f = pd.read_csv(os.path.join(root_dir, 'train_features.csv'))
train_t = pd.read_csv(os.path.join(root_dir, 'train_target.csv'))
test_f = pd.read_csv(os.path.join(root_dir, 'test_features.csv'))

train_f = train_f[['Time','S1','S2','S3','S4']].values
train_f = train_f.reshape((-1, 1, 375, 5))#.astype(np.float32)

test_f = test_f[['Time','S1','S2','S3','S4']].values
test_f = test_f.reshape((-1, 1, 375, 5))#.astype(np.float32)
test_f = torch.FloatTensor(test_f)

### Train

In [5]:
loss_per_model = {}
for name in ['XY','M','V']:
    print('{} train...'.format(name))
    
    # make dataset
    train_target = train_t[list(name)].values#.astype(np.float32)

    # trainx, valx, trainy, valy = train_test_split(train_f, train_target, test_size = 0.2, shuffle = True, random_state = 38)
    fold = KFold(nfold, shuffle = True, random_state= 25)
    loss_per_cv = []
    for i, (train_idx, val_idx) in enumerate(fold.split(train_f, y = train_target)):
        print('fold {}'.format(i+1))
        trainx = train_f[train_idx]
        valx = train_f[val_idx]
        trainy = train_target[train_idx]
        valy = train_target[val_idx]
        
        train_dataset = dfDataset(trainx.astype(np.float32), trainy)
        train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
        val_dataset = dfDataset(valx.astype(np.float32), valy)
        val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

        conv1 = conv_block([8, 16, 32, 64, 128], [1, 375, 5], (3, 1)).cuda() # 128 * 9 * 5
        conv2 = conv_block([8, 16, 32, 64, 128], [1, 375, 5], (4, 1)).cuda() # 128 * 8 * 5
        conv3 = conv_block([8, 16, 32, 64, 128], [1, 375, 5], (5, 1)).cuda() # 128 * 7 * 5
        conv4 = conv_block([8, 16, 32, 64, 128], [1, 375, 5], (6, 1)).cuda() # 128 * 6 * 5

        fc = classifier([128, 64, 32, 16], input_size = 128*30*5, output_size = len(name)).cuda()
        fc = fc.cuda()

        model = cnn_parallel([conv1, conv2, conv3, conv4], fc).cuda()
        
        optimizer = torch.optim.Adam(model.parameters(), lr = base_lr)

        if name == 'XY':
            criterion = E1_loss
        else:
            criterion = E2_loss

        model = model.cuda()
        if initialize:
            model.apply(weights_init)

        curr_loss = 1e+7
        os.makedirs(save_path) if not os.path.exists(save_path) else None
        #train
        for ep in range(1, EPOCH + 1):
            model.train()
            loss = train_model(model, train_loader, criterion, optimizer, criterion)
            model.eval()
            val_loss =eval_model(model, val_loader, criterion)
            if curr_loss > val_loss:
                print('[{}] : train loss {:4f}, val loss drop {:.4f} to {:.4f}'.format(ep, np.mean(loss), curr_loss, val_loss))
                curr_loss = val_loss
                torch.save(model.state_dict(), os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i+1)))
        loss_per_cv.append(curr_loss)
    loss_per_model[name] = loss_per_cv           
            #else:
                #print('[{}] : train loss {:.4f}, val loss {:.4f}, not drop'.format(ep, np.mean(loss), val_loss))

XY train...
fold 1
[1] : train loss 2.373179, val loss drop 10000000.0000 to 1.5837
[2] : train loss 1.157303, val loss drop 1.5837 to 0.8538
[3] : train loss 0.546648, val loss drop 0.8538 to 0.3697
[4] : train loss 0.260515, val loss drop 0.3697 to 0.2026
[5] : train loss 0.138248, val loss drop 0.2026 to 0.1709
[6] : train loss 0.097372, val loss drop 0.1709 to 0.1159
[7] : train loss 0.069203, val loss drop 0.1159 to 0.0800
[8] : train loss 0.053678, val loss drop 0.0800 to 0.0523
[11] : train loss 0.027262, val loss drop 0.0523 to 0.0377
[13] : train loss 0.021589, val loss drop 0.0377 to 0.0331
[15] : train loss 0.016215, val loss drop 0.0331 to 0.0193
[19] : train loss 0.013402, val loss drop 0.0193 to 0.0185
[21] : train loss 0.010613, val loss drop 0.0185 to 0.0171
[26] : train loss 0.005776, val loss drop 0.0171 to 0.0131
[27] : train loss 0.005212, val loss drop 0.0131 to 0.0113
[28] : train loss 0.004552, val loss drop 0.0113 to 0.0110
[30] : train loss 0.005181, val loss d

In [6]:
with open(os.path.join(save_path, 'loss_info.json'), 'w') as f:
    for k in loss_per_model:
        loss_per_model[k] = np.mean(loss_per_model[k])
    f.write(json.dumps(loss_per_model))

## test

In [14]:
submission = pd.read_csv(os.path.join(root_dir, 'sample_submission.csv'))
for name in ['XY','M','V']:
    fc = classifier([128, 64, 32, 16], input_size = 512*3*5, output_size = len(name))
    conv = conv_block([16, 32, 64, 128, 256, 512], [1, 375, 5], (3, 1))
    # define model
    model = cnn_model(conv, fc)
    pred_array = []
    for i in range(1, nfold + 1):
        model.load_state_dict(torch.load(os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i))))
        model = model.cuda()

        with torch.no_grad():
            predict = model(test_f.cuda())
        pred_array.append(predict.detach().cpu().numpy())
    submission[list(name)] = np.mean(pred_array, axis = 0)

In [15]:
submission.head()

,id,X,Y,M,V
0,2800,-261.139008,-39.881229,111.966736,0.434986
1,2801,316.171234,-286.514954,90.317627,0.421709
2,2802,-233.366653,128.657593,28.895294,0.357899
3,2803,160.693039,276.158539,27.653961,0.372838
4,2804,-170.325470,187.950928,133.650543,0.478190


In [16]:
submission.to_csv(os.path.join(save_path, 'submit.csv'), index = False)

In [18]:
x = torch.randn(1, 10)
y = torch.randn(1, 10)